In [2]:
#download python3-discogs-client
import discogs_client
import pandas as pd

In [74]:
consumer_key = 'ykUodTXeeVzqTJkACRzX'
consumer_secret = 'fOWGGoeHRESxkDPVoPFQKJBaTtDrFFRn'

d = discogs_client.Client('CSS_memberfetch/1.0', consumer_key=consumer_key, consumer_secret=consumer_secret)

# Generate the authorize URL
request_token, request_token_secret, authorize_url = d.get_authorize_url()

In [75]:
authorize_url

'https://www.discogs.com/oauth/authorize?oauth_token=gRNZiTzxHsbZluYqrahXDPIRKBbHPMsEGPlfuYgE'

In [ ]:
# After authorization, Discogs will give you a verifier code
verifier = 'VfCCikOYJZ'
# Get the access token
access_token, access_token_secret = d.get_access_token(verifier)
#lad vær med at køre det her mere end 1 gang, så får man fejl.

HTTPError: 401: Invalid response from access token URL.

In [78]:
d.set_token(access_token, access_token_secret)

In [ ]:
#token = "IhdTmHAHnVrjBojtrkYDREYHBTUEBozoQwcpNknH"
#d = discogs_client.Client('Memberfetch/1.0', user_token=token)

In [ ]:
def get_discogs_members(band_name):
    
    if band_name[-7:]==" (band)": # to avoid (band) ending for messing up search results
        band_name = band_name[:-7]
    
    results = d.search(band_name, type='artist') 
    if len(results)>0:
        i = 0
        while i < len(results):
            artist = results[i]
            if band_name.lower() in artist.name.lower():
                break
            i += 1
        
        if i != len(results):
            try:
                if hasattr(artist, 'members'):
                    return [member.name for member in artist.members]
            except discogs_client.exceptions.HTTPError as e:
                pass
    return []
    

In [178]:
get_discogs_members("Alabama Black Snakes")

['Jonas Jessen',
 'Hank Lee Tuffer',
 'Mads Østergaard (2)',
 'Gustav Pontoppidan',
 'Rasmus Christian Larsen']

In [ ]:
# problem: nogle gange står navnene i profile afsnit

In [59]:
df_groups = pd.read_csv("data/groups_wMembers.csv")
df_groups = df_groups.drop("member", axis = 1)

In [185]:
empty_member_indices = df_groups[df_groups["cleaned_member"].apply(lambda x: x == "[]")].index
print(empty_member_indices)
df_groups.loc[empty_member_indices, 'cleaned_member']= df_groups.loc[empty_member_indices, 'title'].apply(get_discogs_members)

Index([], dtype='int64')


In [187]:
df_groups.to_csv("data/groups_wMembers2.csv", index=False)

In [186]:
df_groups

,pageid,title,cleaned_member
0,489818,A Friend in London,['Tim Schou']
1,157530,Ache,"['Per Wium', 'Peter Mellin', 'Finn Olafsson', ..."
2,255559,ADS (band),"['Michael Funder Thorlasius', 'Stig Pedersen',..."
3,976777,Agathorn,[]
4,493993,Alabama Black Snakes,"[Jonas Jessen, Hank Lee Tuffer, Mads Østergaar..."
...,...,...,...
1029,37360,Odense Symfoniorkester,"[Bjarne Hansen, Ulrike Kipp, Michaela Fukačová..."
1030,162784,Sjællands Symfoniorkester,"[Ole Böhn, Harry Gram-Jensen, Fritz O. Jacobse..."
1031,10733,Sønderjyllands Symfoniorkester,"[Anne Augustinsson, Jesper Hindø, Nikolai Skli..."
1032,369172,Aalborg Symfoniorkester,"[Marius Ungureanu, Ib Dirkov, Moshe Atzmon, Vi..."


In [ ]:
# problem: nogle gange er der flere bands med samme navn på discogs (fx the fashion). 
# hvis ikke det danske band er det først, så er det svært at gøre noget ved det

In [ ]:
### problem: discogs har et tal efter personer hvis der er flere med samme navn
# ikke et problemm internt, men hvis det sakl sammenlignes med wiki går det galt
# måske kan vi bare fjerne det og håbe der ikke er flere med samme navn der optræder.